In [111]:
%%capture
!pip install rasterio
!pip install pytorch_model_summary
import rasterio
import rasterio.plot
import os
import cv2
import numpy as np
from numpy import array as array
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pprint import pprint
from sklearn.model_selection import train_test_split
import torch

from google.colab import drive
drive.mount('/content/drive')

In [112]:
image_folder_path = '/content/drive/My Drive/nigeria_rgb/images'
csv_path = '/content/drive/My Drive/nigeria_rgb/nigeria_mpi.csv'
folder_path = '/content/drive/My Drive/nigeria_rgb/'

In [89]:
!ls '/content/drive/My Drive/nigeria_rgb/images'

 Sentinel-2_Abia-0000000000-0000000000.tif
 Sentinel-2_Abia-0000013568-0000000000.tif
 Sentinel-2_Adamawa-0000000000-0000000000.tif
 Sentinel-2_Adamawa-0000000000-0000013568.tif
 Sentinel-2_Adamawa-0000013568-0000000000.tif
 Sentinel-2_Adamawa-0000013568-0000013568.tif
 Sentinel-2_Adamawa-0000027136-0000000000.tif
 Sentinel-2_Adamawa-0000027136-0000013568.tif
'Sentinel-2_Akwa Ibom.tif'
 Sentinel-2_Anambra.tif
 Sentinel-2_Bauchi-0000000000-0000000000.tif
 Sentinel-2_Bauchi-0000000000-0000013568.tif
 Sentinel-2_Bauchi-0000013568-0000000000.tif
 Sentinel-2_Bauchi-0000013568-0000013568.tif
 Sentinel-2_Bauchi-0000027136-0000000000.tif
 Sentinel-2_Bauchi-0000027136-0000013568.tif
 Sentinel-2_Bayelsa.tif
 Sentinel-2_Benue-0000000000-0000000000.tif
 Sentinel-2_Benue-0000000000-0000013568.tif
 Sentinel-2_Benue-0000013568-0000000000.tif
 Sentinel-2_Benue-0000013568-0000013568.tif
 Sentinel-2_Borno-0000000000-0000000000.tif
 Sentinel-2_Borno-0000000000-0000013568.tif
 Sentinel-2_Borno-0000000000-

In [90]:
df = pd.read_csv(csv_path)
df

,region,mpi,pop_in_mpi,deprivation_intensity,vulnerable_to_poverty,in_severe_poverty,pop_share,pop_size,num_mpi_poor,nutrition,child_mortality,years_of_schooling,school_attendance,cooking_fuel,sanitation,drinking_water,electricity,housing,assets
0,Abia,0.035,9.33,38.02,21.41,0.54,1.42,"2,861",267,7.33,3.88,1.22,1.02,7.55,6.72,5.19,2.16,0.37,1.53
1,Adamawa,0.333,62.52,53.24,21.87,35.90,2.12,"4,256","2,661",47.58,10.85,31.08,34.56,62.41,24.74,36.35,39.93,43.46,20.08
2,Akwa Ibom,0.085,20.80,41.11,18.90,3.70,2.15,"4,328",900,12.13,6.64,1.30,4.52,20.17,14.47,15.63,10.66,12.02,7.14
3,Anambra,0.035,8.60,40.30,19.75,1.06,3.32,"6,671",574,5.51,2.14,1.26,1.01,8.28,7.67,4.87,5.14,2.10,4.56
4,Bauchi,0.451,74.80,60.33,15.83,55.26,3.54,"7,120","5,325",56.54,21.59,37.77,48.86,74.19,59.92,37.22,56.10,62.04,28.60
5,Bayelsa,0.117,28.48,41.23,25.57,5.21,0.85,"1,702",485,21.74,4.97,1.02,4.46,25.12,26.65,15.62,19.25,14.43,13.72
6,Benue,0.168,37.17,45.17,30.71,12.27,2.85,"5,729","2,130",24.17,4.33,7.51,12.27,37.13,30.44,14.17,29.80,27.51,18.38
7,Borno,0.342,61.14,56.01,24.30,37.98,3.54,"7,107","4,345",43.22,13.45,33.89,38.59,60.72,37.42,31.55,42.24,29.93,27.04
8,Cross River,0.120,29.03,41.42,25.01,6.58,1.31,"2,628",763,18.87,5.20,4.52,2.13,28.36,26.62,20.66,20.72,15.45,12.41
9,Delta,0.077,20.25,37.83,18.88,1.94,2.22,"4,459",903,12.80,3.08,2.59,3.50,18.21,19.96,11.24,11.47,4.65,6.40


In [91]:
# cleaning dataset
df_numeric = df.loc[ : , df.columns != 'region']

df = df.applymap(lambda s : str(s).replace(",", ""))
df.loc[ : , df.columns != 'region'] =  df.loc[ : , df.columns != 'region'].applymap(lambda s : float(s))

print(len(df))

regions = df["region"]

# df = df.iloc[:2]

df.head()

37


,region,mpi,pop_in_mpi,deprivation_intensity,vulnerable_to_poverty,in_severe_poverty,pop_share,pop_size,num_mpi_poor,nutrition,child_mortality,years_of_schooling,school_attendance,cooking_fuel,sanitation,drinking_water,electricity,housing,assets
0,Abia,0.035,9.33,38.02,21.41,0.54,1.42,2861.0,267.0,7.33,3.88,1.22,1.02,7.55,6.72,5.19,2.16,0.37,1.53
1,Adamawa,0.333,62.52,53.24,21.87,35.9,2.12,4256.0,2661.0,47.58,10.85,31.08,34.56,62.41,24.74,36.35,39.93,43.46,20.08
2,Akwa Ibom,0.085,20.8,41.11,18.9,3.7,2.15,4328.0,900.0,12.13,6.64,1.3,4.52,20.17,14.47,15.63,10.66,12.02,7.14
3,Anambra,0.035,8.6,40.3,19.75,1.06,3.32,6671.0,574.0,5.51,2.14,1.26,1.01,8.28,7.67,4.87,5.14,2.1,4.56
4,Bauchi,0.451,74.8,60.33,15.83,55.26,3.54,7120.0,5325.0,56.54,21.59,37.77,48.86,74.19,59.92,37.22,56.1,62.04,28.6


In [92]:
def sample_images(rgb_image, num_sampled_images, image_length):
    len1, len2 = rgb_image.shape[0], rgb_image.shape[1]
    ys, xs = np.arange((len1 - image_length)//image_length), np.arange((len2 - image_length)//image_length)
    sampled_upper_left_corner_indices = set()
    while len(sampled_upper_left_corner_indices) < num_sampled_images:
        y = np.random.choice(ys) * image_length # corresponding to axis 0
        x = np.random.choice(xs) * image_length # corresponding to axis 1
        if (y, x) in sampled_upper_left_corner_indices:
            continue
        if np.isnan(rgb_image[y:y+image_length, x:x+image_length]).any():
            continue
        sampled_upper_left_corner_indices.add((y, x))
    
    samples = np.zeros((num_sampled_images, image_length, image_length))

    for i, sample in enumerate(sampled_upper_left_corner_indices):
        samples[i] = rgb_image[y:y+image_length, x:x+image_length].copy()
    
    return samples

In [93]:
image_length = 40 # each sampled image is image_length by image_length
num_images_per_region = 12

In [94]:
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear
from torch.nn.functional import relu

from pytorch_model_summary import summary

from torch.utils.data import DataLoader, TensorDataset

class CNNRegressor(torch.nn.Module):
  def __init__(self):
    
    super(CNNRegressor, self).__init__()

    # define convolutional layers
    self.conv_layer1 = Conv2d(1, 8, kernel_size=4, stride=1) # 3 in-channels because of RGB image? nope, let's just do 1 for now
    self.maxpool1 = MaxPool2d(kernel_size=4, stride=2)
    self.conv_layer2 = Conv2d(8, 16, kernel_size=2, stride=2)
    self.maxpool2 = MaxPool2d(kernel_size=4, stride=2)
    self.flatten_layer = Flatten()
    self.linear_layer = Linear(5184, 10)
    self.output_layer = Linear(10, 1)
    self.double()

  # define the method to feed the inputs to the model
  def forward(self, input):
    input = input.double()
    output = relu(self.conv_layer1(input))
    output = relu(self.conv_layer2(output))
    output = self.flatten_layer(output)
    output = relu(self.linear_layer(output))
    output = self.output_layer(output)
    return output

print(summary(CNNRegressor(), torch.zeros((1, 1, image_length, image_length)), show_input=True))

-----------------------------------------------------------------------
      Layer (type)         Input Shape         Param #     Tr. Param #
          Conv2d-1      [1, 1, 40, 40]             136             136
          Conv2d-2      [1, 8, 37, 37]             528             528
         Flatten-3     [1, 16, 18, 18]               0               0
          Linear-4           [1, 5184]          51,850          51,850
          Linear-5             [1, 10]              11              11
Total params: 52,525
Trainable params: 52,525
Non-trainable params: 0
-----------------------------------------------------------------------


In [95]:
class LinearRegressor(torch.nn.Module):
  def __init__(self):
    
    super(LinearRegressor, self).__init__()
    self.output_layer = Linear(1600, 1)
    self.flatten_layer = Flatten()
    self.double()

  # define the method to feed the inputs to the model
  def forward(self, input):
    input = input.double()
    output = self.flatten_layer(input)
    output = self.output_layer(output)
    return output

print(summary(LinearRegressor(), torch.zeros((1, 1, image_length, image_length)), show_input=True))

-----------------------------------------------------------------------
      Layer (type)         Input Shape         Param #     Tr. Param #
         Flatten-1      [1, 1, 40, 40]               0               0
          Linear-2           [1, 1600]           1,601           1,601
Total params: 1,601
Trainable params: 1,601
Non-trainable params: 0
-----------------------------------------------------------------------


In [96]:
%%capture
from torch.optim import Adam
from torch.nn import MSELoss

!pip install pytorch-ignite
from ignite.contrib.metrics.regression.r2_score import R2Score

In [97]:
def model_loss(model, dataset, train = False, optimizer = None):
  loss_fn = MSELoss()
  score_metric = R2Score()

  avg_loss = 0
  avg_score = 0
  count = 0

  all_outputs = []
  all_predictions = []

  for input, output in iter(dataset):
    # get predictions of the model for training set

    input = input.float()
    predictions = model(input)

    # calculate loss of the model
    loss = loss_fn(predictions, output)

    # compute the R2 score
    score_metric.update([predictions, output])
    try:
        score = score_metric.compute()
    except:
        score = 0

    if(train):
      # clear the errors
      optimizer.zero_grad()

      # compute the gradients for optimizer
      loss.backward()

      # use optimizer in order to update parameters
      # of the model based on gradients
      optimizer.step()

    # store the loss and update values
    avg_loss += loss.item()
    avg_score += score
    count += 1

    if not train:
      all_outputs.append(output.detach().numpy())
      all_predictions.append(predictions.detach().numpy())

  if train:
    return avg_loss/count, avg_score/count
  else:
    return avg_loss/count, avg_score/count, np.array(all_predictions).squeeze(), np.array(all_outputs).squeeze()

In [98]:
# BIG for loop
labels_of_interest = ["mpi", "pop_in_mpi", "deprivation_intensity"] + ["nutrition", "child_mortality", "years_of_schooling", "school_attendance", "cooking_fuel", "sanitation", "drinking_water", "electricity", "housing", "assets"]

numpy_file_location = os.path.join(folder_path, "X.npy")
cnn_results_files_location = os.path.join(folder_path, "results_cnn.npy")
lr_results_files_location = os.path.join(folder_path, "results_lr.npy")

with open(numpy_file_location, "rb") as f:
    X = np.load(f)
    loaded = True

for model_name, model in zip(["cnn", "lr"], [CNNRegressor(), LinearRegressor()]):
# for model_name, model in zip(["lr"], [LinearRegressor()]):
    train_predictions = {label_of_interest: [] for label_of_interest in labels_of_interest}
    test_predictions = {label_of_interest: [] for label_of_interest in labels_of_interest}
    train_responses = {label_of_interest: [] for label_of_interest in labels_of_interest}
    test_responses = {label_of_interest: [] for label_of_interest in labels_of_interest}
    train_mse = {label_of_interest: 0 for label_of_interest in labels_of_interest}
    test_mse = {label_of_interest: 0 for label_of_interest in labels_of_interest}
    train_r2 = {label_of_interest: 0 for label_of_interest in labels_of_interest}
    test_r2 = {label_of_interest: 0 for label_of_interest in labels_of_interest}
    results_predicted = {label_of_interest: [] for label_of_interest in labels_of_interest}
    results_actual = {label_of_interest: [] for label_of_interest in labels_of_interest}
    all_preds_labels = {label_of_interest: [] for label_of_interest in labels_of_interest}
    all_outputs_labels = {label_of_interest: [] for label_of_interest in labels_of_interest}

    for label_of_interest in labels_of_interest:

        regions = list(df["region"])
        region_indices = {region: i for i, region in enumerate(regions)}
        # print(region_indices)
        num_regions = len(regions)

        labels = {region: 0 for region in regions}
        for region in regions:
            labels[region] = df[df["region"] == region][label_of_interest]

        # X = np.zeros((num_regions * num_images_per_region, image_length, image_length))
        y = np.zeros(num_regions * num_images_per_region)

        for i, region in enumerate(regions):
            y[num_images_per_region * i : num_images_per_region * (i+1)] = labels[region]

        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=288)

        X_train = X_train[:, None, :, :]
        X_test = X_test[:, None, :, :]

        # convert the values of training set to the numpy array
        try:
            X_train_np = X_train.to_numpy()
            y_train_np = y_train.to_numpy()
        except:
            X_train_np = X_train
            y_train_np = y_train

        # convert the values of testing set to the numpy array
        try:
            X_test_np = X_test.to_numpy()
            y_test_np = y_test.to_numpy()
        except:
            X_test_np = X_test
            y_test_np = y_test

        if torch.cuda.is_available():
            model.cuda() # if GPU is available

        epochs = 100
        optimizer = Adam(model.parameters(), lr = 2e-5)

        if torch.cuda.is_available():
            train_inputs = torch.from_numpy(X_train_np).cuda().double()
            train_outputs = torch.from_numpy(y_train_np.reshape(y_train_np.shape[0],1)).cuda().double()
            test_inputs = torch.from_numpy(X_test_np).cuda().double()
            test_outputs = torch.from_numpy(y_test_np.reshape(y_test_np.shape[0],1)).cuda().double()
        else:
            train_inputs = torch.from_numpy(X_train_np)
            train_outputs = torch.from_numpy(y_train_np.reshape(y_train_np.shape[0],1))
            test_inputs = torch.from_numpy(X_test_np)
            test_outputs = torch.from_numpy(y_test_np.reshape(y_test_np.shape[0],1))

        tensor = TensorDataset(train_inputs, train_outputs)
        loader = DataLoader(tensor, batch_size=1, shuffle = True, drop_last = False)

        for epoch in range(epochs):
            avg_loss, avg_r2_score = model_loss(model, loader, train = True, optimizer = optimizer)
            if epoch % 20 == 0:
                print("Epoch " + str(epoch + 1) + ":\n\tLoss = " + str(avg_loss) + "\n\tR^2 Score = " + str(avg_r2_score))

        torch.save(model, '/content/drive/My Drive/classes/cs288/project/model-{}.pt'.format(label_of_interest))

        tensor = TensorDataset(train_inputs, train_outputs)
        loader = DataLoader(tensor, batch_size=1, shuffle = False, drop_last = True)

        train_avg_loss, train_avg_r2_score, train_preds, train_outputs = model_loss(model, loader, train=False)

        train_mse[label_of_interest] = train_avg_loss
        train_r2[label_of_interest] = train_avg_r2_score
        train_responses[label_of_interest] = train_outputs
        train_predictions[label_of_interest] = train_preds

        tensor = TensorDataset(test_inputs, test_outputs)
        loader = DataLoader(tensor, batch_size=1, shuffle = False, drop_last = True)

        # output of the performance of the model
        test_avg_loss, test_avg_r2_score, test_preds, test_outputs = model_loss(model, loader, train=False)

        test_mse[label_of_interest] = test_avg_loss
        test_r2[label_of_interest] = test_avg_r2_score
        test_responses[label_of_interest] = test_outputs
        test_predictions[label_of_interest] = test_preds

        print("The model's L2 loss is: " + str(avg_loss))
        print("The model's R^2 score is: " + str(avg_r2_score))

        # all predictions
        X_all = X.copy()[:, None, :, :]
        all_inputs = torch.from_numpy(X_all)
        all_outputs = torch.from_numpy(y.reshape(y.shape[0],1))

        tensor = TensorDataset(all_inputs, all_outputs)
        loader = DataLoader(tensor, batch_size=1, shuffle = False, drop_last = True)
        all_avg_loss, all_avg_r2_score, all_preds, all_outputs = model_loss(model, loader, train=False)
        
        result_predicted = {region: 0 for region in regions}
        result_actual = {region: 0 for region in regions}

        for i, region in enumerate(regions):
            predicted = all_preds[12*i:12*(i+1)].mean()
            actual = y[12*i:12*(i+1)].mean()
            result_predicted[region] = predicted
            result_actual[region] = actual

        results_predicted[label_of_interest] = result_predicted
        results_actual[label_of_interest] = result_actual
        all_preds_labels[label_of_interest] = all_preds
        all_outputs_labels[label_of_interest] = all_outputs

    if model_name == "cnn":
        with open(cnn_results_files_location, "wb") as f:
            np.save(f, train_mse)
            np.save(f, train_r2)
            np.save(f, train_responses)
            np.save(f, train_predictions)
            np.save(f, test_mse)
            np.save(f, test_r2)
            np.save(f, test_responses)
            np.save(f, test_predictions)
            np.save(f, results_actual)
            np.save(f, results_predicted)
            np.save(f, all_preds_labels)
            np.save(f, all_outputs_labels)
    elif model_name == "lr":
        with open(lr_results_files_location, "wb") as f:
            np.save(f, train_mse)
            np.save(f, train_r2)
            np.save(f, train_responses)
            np.save(f, train_predictions)
            np.save(f, test_mse)
            np.save(f, test_r2)
            np.save(f, test_responses)
            np.save(f, test_predictions)
            np.save(f, results_actual)
            np.save(f, results_predicted)
            np.save(f, all_preds_labels)
            np.save(f, all_outputs_labels)
    else:
        print("model name not found!!")

Epoch 1:
	Loss = 1.1940015497807261
	R^2 Score = 7779049.151730266
Epoch 21:
	Loss = 0.051795884586880434
	R^2 Score = 2559.235229916839
Epoch 41:
	Loss = 0.023719864831442058
	R^2 Score = -1926.228582812312
Epoch 61:
	Loss = 0.02035604627562355
	R^2 Score = 504.76647809055595
Epoch 81:
	Loss = 0.009604371382626068
	R^2 Score = -120.48967686516417
The model's L2 loss is: 0.011525716119517115
The model's R^2 score is: 36.858453208308504
Epoch 1:
	Loss = 2007.1364767280866
	R^2 Score = -4143587.6321683344
Epoch 21:
	Loss = 500.71454905613103
	R^2 Score = -27243.10718828588
Epoch 41:
	Loss = 321.06998304755916
	R^2 Score = -1220.7176381082475
Epoch 61:
	Loss = 254.99005387023593
	R^2 Score = 0.5815801137048973
Epoch 81:
	Loss = 210.38381071816872
	R^2 Score = 0.7942591615862237
The model's L2 loss is: 185.5413835578067
The model's R^2 score is: 1.9617567867987271
Epoch 1:
	Loss = 307.8406983666383
	R^2 Score = -33301.347404122156
Epoch 21:
	Loss = 106.24888925061157
	R^2 Score = -2278.002

In [99]:
labels_of_interest = ["mpi", "pop_in_mpi", "deprivation_intensity"] + ["nutrition", "child_mortality", "years_of_schooling", "school_attendance", "cooking_fuel", "sanitation", "drinking_water", "electricity", "housing", "assets"]

In [145]:
from IPython.display import display

def mse(predicted, actual):
    return np.mean((predicted - actual)**2)

for model_name in ["lr", "cnn"]:
    if model_name == "lr":
        lr_results_files_location = os.path.join(folder_path, "results_lr.npy")
        with open(lr_results_files_location, "rb") as f:
            train_mse = np.load(f, allow_pickle=True)
            train_r2 = np.load(f, allow_pickle=True)
            train_responses = np.load(f, allow_pickle=True)
            train_predictions = np.load(f, allow_pickle=True)
            test_mse = np.load(f, allow_pickle=True)
            test_r2 = np.load(f, allow_pickle=True)
            test_responses = np.load(f, allow_pickle=True)
            test_predictions = np.load(f, allow_pickle=True)
            results_actual = np.load(f, allow_pickle=True)
            results_predicted = np.load(f, allow_pickle=True)
            all_preds_labels = np.load(f, allow_pickle=True)
            all_outputs_labels = np.load(f, allow_pickle=True)
    elif model_name == "cnn":
        cnn_results_files_location = os.path.join(folder_path, "results_cnn.npy")
        with open(cnn_results_files_location, "rb") as f:
            train_mse = np.load(f, allow_pickle=True)
            train_r2 = np.load(f, allow_pickle=True)
            train_responses = np.load(f, allow_pickle=True)
            train_predictions = np.load(f, allow_pickle=True)
            test_mse = np.load(f, allow_pickle=True)
            test_r2 = np.load(f, allow_pickle=True)
            test_responses = np.load(f, allow_pickle=True)
            test_predictions = np.load(f, allow_pickle=True)
            results_actual = np.load(f, allow_pickle=True)
            results_predicted = np.load(f, allow_pickle=True)
            all_preds_labels = np.load(f, allow_pickle=True)
            all_outputs_labels = np.load(f, allow_pickle=True)

    print(f"{model_name}")
    mpi_vals = np.array(list(results_actual[()]["mpi"].values()))

    print("direct_method")
    direct_method = np.array(list(results_predicted[()]["mpi"].values()))
    print(mse(direct_method, mpi_vals))
    display(direct_method)

    print("factor method, AF")
    factor_average_first = np.array(list(results_predicted[()]["pop_in_mpi"].values())) * np.array(list(results_predicted[()]["deprivation_intensity"].values())) / 10000
    print(mse(factor_average_first, mpi_vals))
    display(factor_average_first)

    print("factor method, AL")
    average_last_predictions = all_preds_labels[()]["pop_in_mpi"] * all_preds_labels[()]["deprivation_intensity"] / 10000
    factor_average_last = np.zeros(37)
    for i in range(37):
        for j in range(12):
            factor_average_last[i] = np.mean(average_last_predictions[12*i:12*(i+1)])

    print(mse(factor_average_last, mpi_vals))
    display(factor_average_last)

    print("components method, AF")
    components_average_first = np.zeros(37)
    for label in ["nutrition", "child_mortality", "years_of_schooling", "school_attendance"]:
        components_average_first += np.array(list(results_predicted[()][label].values()))/6
    for label in ["cooking_fuel", "sanitation", "drinking_water", "electricity", "housing", "assets"]:
        components_average_first += np.array(list(results_predicted[()][label].values()))/18
    components_average_first/=100

    print(mse(components_average_first, mpi_vals))
    display(components_average_first)

    print("components method, AL")
    average_last_component_predictions = np.zeros(444)
    for label in ["nutrition", "child_mortality", "years_of_schooling", "school_attendance"]:
        average_last_component_predictions += np.array(list(all_preds_labels[()][label]))/6
    for label in ["cooking_fuel", "sanitation", "drinking_water", "electricity", "housing", "assets"]:
        average_last_component_predictions += np.array(list(all_preds_labels[()][label]))/18
    average_last_component_predictions/=100

    components_average_last = np.zeros(37)
    for i in range(37):
        for j in range(12):
            components_average_last[i] = np.mean(average_last_component_predictions[12*i:12*(i+1)])

    print(mse(components_average_last, mpi_vals))
    display(components_average_last)

lr
direct_method
142.1041103088943


array([-2.27829305e+01, -9.58082547e+00, -1.84761764e+01,  1.02915309e+01,
       -1.40291041e+01,  4.29893167e+00, -8.21780933e+00, -1.44729467e+01,
        1.93763965e+00, -9.66666395e+00, -4.70429600e+00, -1.76769879e+01,
       -1.27777239e+01, -1.63922060e+01, -9.37845472e+00,  1.78125711e+00,
       -1.32243783e+01, -7.74065301e+00, -1.60161020e+01, -1.85030410e+01,
       -1.25317789e+01,  1.22493148e+00, -2.86866957e+00, -1.07038396e+01,
       -1.22651277e+01, -1.21067758e+01, -7.22925505e+00, -5.35891271e+00,
       -6.97728018e+00, -1.68710169e+01, -7.88963419e+00, -8.65658364e+00,
       -1.29975725e+01, -1.10084697e+01, -1.42513903e+01, -1.19509528e+01,
        3.91924059e-03])

factor method, AF
0.05577358052643354


array([-3.40915631e-02,  1.40026031e-01,  1.54172990e-03,  4.73002964e-02,
        8.50744501e-02,  3.32762319e-02,  4.88284427e-02, -3.58677240e-02,
        9.53071028e-03, -6.11714174e-02,  5.60289189e-02, -1.46853234e-02,
       -3.35361836e-02, -3.35538866e-03, -7.05456647e-02,  2.43857390e-01,
       -1.97075007e-01,  2.57680458e-01,  9.52383227e-03,  6.81881410e-02,
       -5.80617610e-03,  1.72035360e-01, -3.03381916e-02, -7.61471866e-05,
       -1.07454505e-01,  6.43687582e-03,  1.69080881e-01, -5.45397740e-02,
       -5.29155653e-02, -6.81915400e-02, -9.51775680e-02,  1.94714463e-02,
       -1.87056812e-01,  2.81137969e-01,  7.96724573e-03,  1.96460814e-01,
        6.98035670e-07])

factor method, AL
6.492574148095369


...

array([0.42735415, 0.70340362, 0.4208051 , 0.34641522, 0.67761287,
       0.38992909, 0.65648374, 0.43518952, 0.43863811, 0.20575132,
       0.33793716, 0.59166086, 0.43694484, 0.36451591, 0.42162679,
       0.5815232 , 0.17756284, 0.78673693, 0.4339482 , 0.60061233,
       0.50026448, 0.75124431, 0.45685314, 0.52495679, 0.32284059,
       0.60297197, 0.87001201, 0.38988055, 0.13511699, 0.2710026 ,
       0.16584882, 0.46250449, 0.48330474, 0.94326486, 0.8111475 ,
       0.84316127, 0.34016252])

components method, AF
2.2261286258754916e-05


array([0.03949295, 0.33379645, 0.08593318, 0.03377162, 0.45430319,
       0.11174179, 0.16902147, 0.34833606, 0.1225747 , 0.06885857,
       0.19487812, 0.09161108, 0.08770451, 0.08730718, 0.12181999,
       0.4863129 , 0.03771746, 0.4947615 , 0.31055292, 0.371607  ,
       0.39703716, 0.58805958, 0.15095766, 0.21049499, 0.0136451 ,
       0.15649394, 0.38023107, 0.04855184, 0.08701899, 0.08193644,
       0.08387068, 0.25011922, 0.04721194, 0.55774302, 0.37346441,
       0.54292569, 0.45889963])

components method, AL
0.0765708268644565


array([0.18891213, 0.75860348, 0.3317577 , 0.08573046, 0.5793049 ,
       0.556244  , 0.30958105, 0.47066605, 0.34189758, 0.0896886 ,
       0.48953882, 0.54802435, 0.18786275, 0.19264692, 0.5246621 ,
       0.6942476 , 0.23550939, 0.65963077, 0.70340055, 0.42858056,
       0.4273517 , 0.81720786, 0.44117157, 0.56902101, 0.35972461,
       0.24702468, 0.7365099 , 0.14808007, 0.52732166, 0.35263246,
       0.54336094, 0.58963961, 0.2952793 , 0.88183551, 0.48114889,
       0.77842076, 0.8829061 ])

In [150]:
for model_name in ["lr", "cnn"]:
    if model_name == "lr":
        lr_results_files_location = os.path.join(folder_path, "results_lr.npy")
        with open(lr_results_files_location, "rb") as f:
            train_mse = np.load(f, allow_pickle=True)
            train_r2 = np.load(f, allow_pickle=True)
            train_responses = np.load(f, allow_pickle=True)
            train_predictions = np.load(f, allow_pickle=True)
            test_mse = np.load(f, allow_pickle=True)
            test_r2 = np.load(f, allow_pickle=True)
            test_responses = np.load(f, allow_pickle=True)
            test_predictions = np.load(f, allow_pickle=True)
            results_actual = np.load(f, allow_pickle=True)
            results_predicted = np.load(f, allow_pickle=True)
            all_preds_labels = np.load(f, allow_pickle=True)
            all_outputs_labels = np.load(f, allow_pickle=True)
    elif model_name == "cnn":
        cnn_results_files_location = os.path.join(folder_path, "results_cnn.npy")
        with open(cnn_results_files_location, "rb") as f:
            train_mse = np.load(f, allow_pickle=True)
            train_r2 = np.load(f, allow_pickle=True)
            train_responses = np.load(f, allow_pickle=True)
            train_predictions = np.load(f, allow_pickle=True)
            test_mse = np.load(f, allow_pickle=True)
            test_r2 = np.load(f, allow_pickle=True)
            test_responses = np.load(f, allow_pickle=True)
            test_predictions = np.load(f, allow_pickle=True)
            results_actual = np.load(f, allow_pickle=True)
            results_predicted = np.load(f, allow_pickle=True)
            all_preds_labels = np.load(f, allow_pickle=True)
            all_outputs_labels = np.load(f, allow_pickle=True)

    print(model_name)
    for label in labels_of_interest:
        # print(label)
        print(test_mse[()][label])

lr
130.73434900937383
2036.9282942269233
164.21598780113146
122.8977347983951
10.861402675583019
350.5863541635906
386.5984884102161
235.50369551344068
297.4970766703586
99.03527602006842
212.601618493474
344.52420875264954
187.4285649327744
cnn
0.016134514461795535
289.14764705191226
107.42775613006776
5.565854112886266
2.1005522415724536
0.3830992624557793
0.470541355792659
0.7327853903819934
0.10127532122413642
0.015875883743658694
0.02876217195036525
0.5599694215921964
3.574585341850319


array({'mpi': 0.016134514461795535, 'pop_in_mpi': 289.14764705191226, 'deprivation_intensity': 107.42775613006776, 'nutrition': 5.565854112886266, 'child_mortality': 2.1005522415724536, 'years_of_schooling': 0.3830992624557793, 'school_attendance': 0.470541355792659, 'cooking_fuel': 0.7327853903819934, 'sanitation': 0.10127532122413642, 'drinking_water': 0.015875883743658694, 'electricity': 0.02876217195036525, 'housing': 0.5599694215921964, 'assets': 3.574585341850319},
      dtype=object)

In [141]:
average_last_predictions = all_preds_labels[()]["pop_in_mpi"] * all_preds_labels[()]["deprivation_intensity"] / 10000
factor_average_last = np.zeros(37)
for i in range(37):
    for j in range(12):
        factor_average_last[i] = np.mean(average_last_predictions[12*i:12*(i+1)])

print(np.mean((mpi_vals - factor_average_last)**2))

factor_average_last

0.05577358052643354


array([-3.40915631e-02,  1.40026031e-01,  1.54172990e-03,  4.73002964e-02,
        8.50744501e-02,  3.32762319e-02,  4.88284427e-02, -3.58677240e-02,
        9.53071028e-03, -6.11714174e-02,  5.60289189e-02, -1.46853234e-02,
       -3.35361836e-02, -3.35538866e-03, -7.05456647e-02,  2.43857390e-01,
       -1.97075007e-01,  2.57680458e-01,  9.52383227e-03,  6.81881410e-02,
       -5.80617610e-03,  1.72035360e-01, -3.03381916e-02, -7.61471866e-05,
       -1.07454505e-01,  6.43687582e-03,  1.69080881e-01, -5.45397740e-02,
       -5.29155653e-02, -6.81915400e-02, -9.51775680e-02,  1.94714463e-02,
       -1.87056812e-01,  2.81137969e-01,  7.96724573e-03,  1.96460814e-01,
        6.98035670e-07])

In [138]:
mpi_vals = np.array(list(results_predicted[()]["mpi"].values()))

np.mean((mpi_vals - np.array(list(results_actual[()]["mpi"].values())))**2)

np.array(list(results_actual[()]["mpi"].values()))

array([0.035, 0.333, 0.085, 0.035, 0.451, 0.117, 0.168, 0.342, 0.12 ,
       0.077, 0.194, 0.09 , 0.087, 0.087, 0.119, 0.485, 0.049, 0.493,
       0.308, 0.369, 0.398, 0.585, 0.148, 0.21 , 0.016, 0.161, 0.378,
       0.049, 0.087, 0.085, 0.085, 0.255, 0.056, 0.555, 0.369, 0.54 ,
       0.476])

In [129]:
factor_average_first = np.array(list(results_predicted[()]["pop_in_mpi"].values())) * np.array(list(results_predicted[()]["deprivation_intensity"].values())) / 10000

np.mean((mpi_vals - factor_average_first)**2)

137.98274302034088

In [130]:
components_average_first = np.zeros(37)
for label in ["nutrition", "child_mortality", "years_of_schooling", "school_attendance"]:
    components_average_first += np.array(list(results_predicted[()][label].values()))/6
for label in ["cooking_fuel", "sanitation", "drinking_water", "electricity", "housing", "assets"]:
    components_average_first += np.array(list(results_predicted[()][label].values()))/18
components_average_first/=100

np.mean((mpi_vals - components_average_first)**2)

142.36483476324292

In [134]:
average_last_component_predictions = np.zeros(444)
for label in ["nutrition", "child_mortality", "years_of_schooling", "school_attendance"]:
    average_last_component_predictions += np.array(list(all_preds_labels[()][label]))/6
for label in ["cooking_fuel", "sanitation", "drinking_water", "electricity", "housing", "assets"]:
    average_last_component_predictions += np.array(list(all_preds_labels[()][label]))/18
average_last_component_predictions/=100

components_average_last = np.zeros(37)
for i in range(37):
    for j in range(12):
        components_average_last[i] = np.mean(average_last_component_predictions[12*i:12*(i+1)])

np.mean((mpi_vals - components_average_last)**2)

197.1632117355797

In [105]:
all_outputs.shape

(444,)

In [106]:
np.mean((all_preds[()]["pop_in_mpi"] * test_responses[()]["deprivation_intensity"] / 10000 - test_responses[()]["mpi"])**2)

IndexError: ignored

In [ ]:
print(test_predictions[()]["mpi"])

In [ ]:
print(np.mean((test_responses[()]["mpi"] - test_predictions[()]["mpi"])**2))

In [ ]:
for label_of_interest in labels_of_interest:
    print(np.round(np.mean((test_responses[()][label_of_interest] - test_predictions[()][label_of_interest])**2), 3))

In [ ]:
# print(results_predicted[()]["mpi"])

# results_mpi_direct_vals = np.array(list(results_predicted[()]["mpi"].values()))
# results_mpi_direct_vals

# print(np.mean((results_mpi_direct_vals - results_mpi_vals)**2))

In [ ]:
results_mpi_indirect = {region:0 for region in regions}
for region in regions:
    results_mpi_indirect[region] = results_predicted[()]["pop_in_mpi"][region] * results_predicted[()]["deprivation_intensity"][region] / 10000

print(results_mpi_indirect)

results_mpi_indirect_vals = np.array(list(results_mpi_indirect.values()))
results_mpi_indirect_vals

print(np.mean((results_mpi_indirect_vals - results_mpi_vals)**2))

In [ ]:
results_actual[()]["mpi"]

print(results_actual[()]["mpi"])

results_mpi_vals = np.array(list(results_actual[()]["mpi"].values()))
results_mpi_vals